In [1]:
#Import Libraries and Data
import numpy as np
import pandas as pd
from scipy.stats import rv_discrete
from sklearn.feature_extraction.text import CountVectorizer

shakespeare=pd.read_csv('../data/external/Shakespeare_data.csv')
shakespeare.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
0,1,Henry IV,NaN,NaN,NaN,ACT I
1,2,Henry IV,NaN,NaN,NaN,SCENE I. London. The palace.
2,3,Henry IV,NaN,NaN,NaN,"Enter KING HENRY, LORD JOHN OF LANCASTER, the ..."
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"


In [2]:
#Clean Data
shakespeare=shakespeare.dropna(subset=['Player'])
shakespeare=shakespeare.dropna(subset=['PlayerLine'])

shakespeare.head()

,Dataline,Play,PlayerLinenumber,ActSceneLine,Player,PlayerLine
3,4,Henry IV,1.0,1.1.1,KING HENRY IV,"So shaken as we are, so wan with care,"
4,5,Henry IV,1.0,1.1.2,KING HENRY IV,"Find we a time for frighted peace to pant,"
5,6,Henry IV,1.0,1.1.3,KING HENRY IV,And breathe short-winded accents of new broils
6,7,Henry IV,1.0,1.1.4,KING HENRY IV,To be commenced in strands afar remote.
7,8,Henry IV,1.0,1.1.5,KING HENRY IV,No more the thirsty entrance of this soil


In [3]:
#----------------------------Define the Hidden States----------------------------
hiddenstates=shakespeare.ActSceneLine.unique()

#Auto Generate the transitions between these states
initialstateprobs=pd.DataFrame(np.random.uniform(size=(len(hiddenstates),len(hiddenstates))),columns=hiddenstates, index=hiddenstates)
initialstateprobs=np.exp(-initialstateprobs)/np.sum(a=initialstateprobs,axis=1)
initialstateprobs.to_csv('../data/internal/hiddenprob.csv',index=False)
initialstateprobs.head()

,1.1.1,1.1.2,1.1.3,1.1.4,1.1.5,1.1.6,1.1.7,1.1.8,1.1.9,1.1.10,...,4.4.920,4.4.921,4.4.922,4.4.923,4.4.924,4.4.925,4.4.926,4.4.927,4.4.928,4.4.929
1.1.1,0.000058,0.000053,0.000062,0.000076,0.000050,0.000059,0.000062,0.000092,0.000068,0.000083,...,0.000082,0.000082,0.000046,0.000076,0.000073,0.000067,0.000061,0.000072,0.000065,0.000122
1.1.2,0.000057,0.000112,0.000088,0.000073,0.000048,0.000073,0.000070,0.000097,0.000084,0.000094,...,0.000087,0.000088,0.000055,0.000111,0.000092,0.000050,0.000068,0.000066,0.000057,0.000072
1.1.3,0.000093,0.000090,0.000114,0.000113,0.000051,0.000122,0.000055,0.000050,0.000087,0.000056,...,0.000054,0.000046,0.000052,0.000066,0.000086,0.000113,0.000115,0.000086,0.000069,0.000076
1.1.4,0.000067,0.000079,0.000058,0.000080,0.000070,0.000085,0.000119,0.000098,0.000047,0.000057,...,0.000061,0.000075,0.000098,0.000090,0.000052,0.000083,0.000104,0.000065,0.000069,0.000109
1.1.5,0.000085,0.000102,0.000104,0.000086,0.000050,0.000056,0.000122,0.000122,0.000049,0.000053,...,0.000062,0.000112,0.000056,0.000047,0.000064,0.000054,0.000070,0.000085,0.000051,0.000052


In [4]:
#-------------------------Observable state calculation----------------------------
obsstates=shakespeare[['ActSceneLine','PlayerLine']]
obsstates['PlayerLine2']=obsstates.PlayerLine.str.replace(pat='[,.:]', repl='').astype(str)
#Figure out initial probabilities through frequency
vectorizer = CountVectorizer()
corpus=obsstates['PlayerLine2'].values
X = vectorizer.fit_transform(corpus)
X.toarray() #To grab this value, vectorizer.get_feature_names()
observablestates=vectorizer.get_feature_names()
print(len(observablestates))

C:\Users\kjadmin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


22572


In [5]:
# need to auto generate our prob and emmission matrix dealy
b_df = pd.DataFrame(np.random.uniform(size=(len(hiddenstates),len(observablestates))),columns=observablestates, index=hiddenstates)
b_df.head()

,10,2d,2s,4d,5s,6d,8d,aaron,abaissiez,abandon,...,zenelophon,zenith,zephyrs,zir,zo,zodiac,zodiacs,zone,zounds,zwaggered
1.1.1,0.777246,0.563671,0.702259,0.311323,0.716067,0.918960,0.292538,0.556490,0.633769,0.678578,...,0.678344,0.363450,0.995435,0.025821,0.885324,0.623894,0.931276,0.358282,0.845359,0.136684
1.1.2,0.338311,0.095942,0.095999,0.264706,0.230192,0.849033,0.597040,0.077879,0.093514,0.918824,...,0.148247,0.176618,0.715617,0.283863,0.929796,0.900524,0.515902,0.267444,0.094218,0.347083
1.1.3,0.925411,0.200073,0.718286,0.330035,0.204939,0.926525,0.890798,0.792723,0.930557,0.445629,...,0.942280,0.371790,0.344330,0.753144,0.134442,0.347286,0.168596,0.082722,0.442660,0.063983
1.1.4,0.131342,0.676649,0.196043,0.692845,0.014944,0.026922,0.972801,0.313374,0.098543,0.263830,...,0.848425,0.749084,0.658452,0.503950,0.140602,0.009890,0.340675,0.445445,0.870999,0.413545
1.1.5,0.057211,0.144307,0.119252,0.379111,0.752851,0.654915,0.131781,0.093556,0.564610,0.723098,...,0.067253,0.805218,0.070903,0.680431,0.905279,0.120630,0.674746,0.020146,0.147125,0.670710


In [6]:
#Transform to probabilities
b_df=np.exp(-b_df)

In [7]:
b_df['sum']= b_df.sum(axis=1)
print(b_df.head())

             10        2d        2s        4d        5s        6d        8d  \
1.1.1  0.459670  0.569116  0.495465  0.732477  0.488670  0.398934  0.746367   
1.1.2  0.712973  0.908517  0.908465  0.767431  0.794381  0.427828  0.550439   
1.1.3  0.396369  0.818671  0.487587  0.718899  0.814697  0.395927  0.410328   
1.1.4  0.876918  0.508318  0.821977  0.500151  0.985167  0.973438  0.378023   
1.1.5  0.944395  0.865622  0.887584  0.684470  0.471022  0.519486  0.876533   

          aaron  abaissiez   abandon    ...      zenelophon    zenith  \
1.1.1  0.573217   0.530588  0.507338    ...        0.507457  0.695273   
1.1.2  0.925076   0.910725  0.398988    ...        0.862218  0.838100   
1.1.3  0.452611   0.394334  0.640421    ...        0.389738  0.689499   
1.1.4  0.730977   0.906157  0.768104    ...        0.428089  0.472799   
1.1.5  0.910687   0.568582  0.485246    ...        0.934958  0.446990   

        zephyrs       zir        zo    zodiac   zodiacs      zone    zounds  \
1.1.1  

In [9]:
df_new = b_df.iloc[:,:-1].div(b_df["sum"], axis=0)

In [12]:
df_new.to_csv('../data/internal/observprob.csv',index=False)

## References
Various Pandas and Matrix Code Snippets
* http://www.blackarbs.com/blog/introduction-hidden-markov-models-python-networkx-sklearn/2/9/2017
* https://stackoverflow.com/questions/25292838/applying-regex-to-a-pandas-dataframe 
* https://stackoverflow.com/questions/48474442/python-from-list-of-list-of-tokens-to-bag-of-words?rq=1
* https://stackoverflow.com/questions/47297585/building-a-transition-matrix-using-words-in-python-numpy
* https://machinelearningmastery.com/prepare-text-data-machine-learning-scikit-learn/
* https://datascience.stackexchange.com/questions/37329/how-to-convert-an-array-of-numbers-into-probability-values
* https://stackoverflow.com/questions/26537878/pandas-sum-across-columns-and-divide-each-cell-from-that-value

## Unused Code Snippets
#Calculate Initial Probabilities for observation states (words) and hidden states (player)

#First, observation states
obsstates=shakespeare[['Player','PlayerLine']]
#Parse out player line
obsstates['PlayerLine2']=obsstates.PlayerLine.str.replace(pat='[,.:]', repl='').astype(str)

#Figure out initial probabilities through frequency
vectorizer = CountVectorizer()
corpus=obsstates['PlayerLine2'].values
X = vectorizer.fit_transform(corpus)
X.toarray() #To grab this value, vectorizer.get_feature_names()
toDivide=len(vectorizer.get_feature_names())
initProb=X/toDivide
print(initProb.toarray())
print(vectorizer.get_feature_names())
